In [43]:
import json
import pandas as pd
import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
from recordlinkage.index import Block
import numpy as np
import math

import mitosheet

In [117]:
sourcefile=r"ODBiz_Source.json"
with open(sourcefile) as source_f:
    Source=json.load(source_f)
    
DF=pd.read_csv(Source["filename"],
               encoding=Source["encoding"],
               index_col=Source["index"], low_memory=False, dtype='str')

#reduce database to only the columns we use for comparisons
DF=DF[Source["column_map"].values()]
column_map={val: key for key, val in Source["column_map"].items()}
DF=DF.rename(columns=column_map)
df=DF.copy()

In [118]:
# FORMATTTING

#remove accents

text_cols=['Name','Address','StreetName','City']
for col in text_cols:
    
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)

#remove periods, apostrophes, commas, and hypens in the Name and address columns

r_list=[r".",r",",r"'",r"-"]

for r in r_list:
    
    df["Name"] = df["Name"].str.replace(r,' ',regex=False)
    df["Address"] = df["Address"].str.replace(r,' ',regex=False)

#remove excess whitespace
df["Name"] = df["Name"].str.replace(r" +"," ",regex=True)
df["Address"] = df["Address"].str.replace(r" +"," ",regex=True)

#standardise postal codes - just remove empty space and make sure it's all lower case

df.loc[~df.PostalCode.isnull(),'PostalCode'] = df.loc[~df.PostalCode.isnull(),'PostalCode'].str.replace(' ','').str.lower()


#Some records have street number and street name, but no address field filled

df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'Address']\
    = clean(df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetNumber']+' '+\
           df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetName']+' '+\
        df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'City'])


In [119]:
len(df)

558964

In [120]:
# Remove those with no business name 

print('Number removed with business name = NA: ', len(df[df.Name.isnull()]))
df = df[~df.Name.isnull()]

Number removed with business name = NA:  29111


In [121]:
# Remove those with inactive status

no_removed = len(df[df.Status.isin(['Gone Out of Business', 'Inactive', 'Cancelled'])])

print('Number removed with inactive status: ', no_removed)
df = df[~df.Status.isin(['Gone Out of Business', 'Inactive', 'Cancelled'])]


Number removed with inactive status:  58598


In [131]:
# Carry out basic deduplication on 
# Exact match for business name, license number, street number, 
# postal code - keep one record


df['dupe_1'] = df.duplicated(subset=['Name','licence_number', 'PostalCode', 'StreetNumber'],keep=False)
df_dup_1 = df.loc[df.dupe_1==True]

df_remove = df.duplicated(subset=['Name','licence_number', 'PostalCode', 'StreetNumber'],keep='last')

# df.loc[df.dupe==True].to_csv('outputs/simple_dupes.csv',encoding='utf-8')
df = df.drop_duplicates(subset=['Name','Address','CSDUID', 'licence_number'],keep='last')

print('No removed with matching name, license, street no and pcode: ', len(df_remove) - len(df))


No removed with matching name, license, street no and pcode:  0


In [130]:
df['dupe_2'] = df.duplicated(subset=['Name', 'PostalCode', 'StreetNumber'],keep=False)
df_dup_2 = df.loc[df.dupe_2==True]

df_remove = df.duplicated(subset=['Name', 'PostalCode', 'StreetNumber'],keep='last')

print('number duplicated on these that would be removed: ', len(df_remove))

number duplicated on these that would be removed:  443450


In [126]:
len(df_dup_2)

63839

In [129]:
mitosheet.sheet(df_dup_2, analysis_to_replay="id-muenxbzusu")

MitoWidget(analysis_data_json='{"analysisName": "id-wotuiqsszq", "analysisToReplay": {"analysisName": "id-muen…

In [ ]:
from mitosheet import *; register_analysis("id-muenxbzusu");
    
# Sorted Name in ascending order
df_dup_2 = df_dup_2.sort_values(by='Name', ascending=True, na_position='first')

# Sorted Name in ascending order
df_dup_2 = df_dup_2.sort_values(by='Name', ascending=True, na_position='first')

# Sorted PostalCode in ascending order
df_dup_2 = df_dup_2.sort_values(by='PostalCode', ascending=True, na_position='first')

# Sorted Name in ascending order
df_dup_2 = df_dup_2.sort_values(by='Name', ascending=True, na_position='first')

# Sorted Name in ascending order
df_dup_2 = df_dup_2.sort_values(by='Name', ascending=True, na_position='first')


In [71]:
# How many also have a postcode that matches elsewhere?

df_post = df_dup[~df_dup.PostalCode.isnull()]

df_post['dupe_code'] = df_post.duplicated(subset=['PostalCode', 'StreetNumber'], keep=False)
df_dup2 = df_post.loc[df_post.dupe_code==True]

<ipython-input-71-3f97d9008d09>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_post['dupe_code'] = df_post.duplicated(subset=['PostalCode', 'StreetNumber'],keep=False)


In [103]:
len(DF)

558964

In [ ]:
r"""
II. Record Linkage


This is the section that uses the record linkage package to determine candidate pairs,
which will be evaluated separately.
"""

In [17]:
# For TESTING

df_input = df
df = df_dup.head(1000)

In [18]:
len(df)

1000

In [19]:
indexer = rl.Index()

In [20]:
# Choose columns to block on

# Try CSDUID to start, but we also need to remove NA

indexer.add(Block('CSDUID'))

In [21]:
df = df[df['CSDUID'].notna()]

In [22]:
candidate_links = indexer.index(df)


In [23]:
len(df)

1000

In [24]:
print (len(df)**2, len(candidate_links))

1000000 499500


In [25]:
print('Computing metrics for {} candidate pairs'.format(len(candidate_links)))

Computing metrics for 499500 candidate pairs


In [26]:
#likely to be a lot of records to match, so split into chunks
n=math.ceil(len(candidate_links)/1E5)
chunks=rl.index_split(candidate_links, n)
# Comparison step
results=[]
#n_jobs specifies number of cores for running in parallel
compare = rl.Compare(n_jobs=8) 
    
compare.exact('StreetNumber', 'StreetNumber', label='StrNum_Match')
compare.exact('PostalCode', 'PostalCode', label='PC_Match')

compare.string('Address', 'Address', method='cosine',  label='Addr_CS')
compare.string('Address', 'Address', method='lcs',  label='Addr_LCS')

compare.string('StreetName', 'StreetName', method='damerau_levenshtein',  label='StrName_DL')
compare.string('StreetName', 'StreetName', method='cosine',  label='StrName_CS')
    
compare.string('City', 'City',  method='damerau_levenshtein',label='City_DL')
compare.string('Name', 'Name',  method='damerau_levenshtein',label='Name_DL')
compare.string('Name', 'Name',  method='cosine',label='Name_CS')

    
i=0
for chunk in chunks:
    i+=1
    print('processing chunk {} of {}'.format(i,n))

    
    features = compare.compute(chunk, df)

    #reduce comparison matrix to entries where the name score is reasonably high

    cutoff=0.9
    features=features.loc[features.Name_CS>cutoff]
    results.append(features)
f=pd.concat(results)
print('Score cut-off of {} reduced candidate pairs to {}'.format(cutoff,len(f)))

f['idx1']=f.index.get_level_values(0)
f['idx2']=f.index.get_level_values(1)

print('Merging on original dataframe and computing distance.')
f=f.merge(df,left_on='idx1',how='left',right_on='idx')

f=f.merge(df,left_on='idx2',how='left',right_on='idx', suffixes=('_1','_2'))

f['Distance']=np.nan

# f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull()),'Distance']=f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull())].apply(lambda row: haversine(row), axis=1)

f=f[['idx1',
     'idx2',
     'Name_1',
     'Name_2',
     'Name_DL',
     'Name_CS',
     'Addr_CS',
     'Addr_LCS',
     'Address_1',
     'Address_2',
     'StrNum_Match',
     'StrName_DL',
     'StrName_CS',
     'PC_Match',
     'Distance']]

f.to_csv('{}'.format(Source["output_name"]),index=False,encoding='cp1252')




processing chunk 1 of 5
processing chunk 2 of 5
processing chunk 3 of 5
processing chunk 4 of 5
processing chunk 5 of 5
Score cut-off of 0.9 reduced candidate pairs to 724
Merging on original dataframe and computing distance.


In [27]:
len(f)

724